In [ ]:
%pip install -r requirements.txt

In [ ]:
data_medals_path = 'data/olympic_medals.xlsx'
data_athletes_path = 'data/olympic_athletes.json'
data_results_path = 'data/olympic_results.html'
data_hosts_path = 'data/olympic_hosts.xml'

In [ ]:
import pandas as pd

tmp_medals = pd.read_excel(data_medals_path)
tmp_athletes = pd.read_json(data_athletes_path)
tmp_results = pd.read_html(data_results_path)
tmp_hosts = pd.read_xml(data_hosts_path)

In [ ]:
df_medals = tmp_medals.copy()
df_athletes = tmp_athletes.copy()
df_results = tmp_results.copy()
df_hosts = tmp_hosts.copy()

In [ ]:
df_medals.head()

In [ ]:
# if athlete_medals is null, fill by 0
df_athletes['athlete_medals'] = df_athletes['athlete_medals'].fillna(0)

In [ ]:
df_athletes.head(5)
df_athletes.isnull().sum()

# first_game laisser tel quel pour voir
# athlete_year_birth laisser tel quel pour voir

In [ ]:
df_results[0].head()

In [ ]:
df_hosts.head()

In [ ]:
# get all countries using df_medals
countries = df_medals['country_name'].unique()
print(len(countries), countries[0:5])

In [ ]:
# get all medal type using df_medals
medal_types = df_medals['medal_type'].unique()
print(medal_types)

In [ ]:
# get all locations using df_athletes
locations = df_athletes['first_game'].unique()
print(len(locations), locations[0:5])


In [ ]:
# get all athletes using df_athletes
athletes = df_athletes['athlete_full_name'].unique()
print(len(athletes), athletes[0:5])

In [ ]:
# get all disciplines using df_athletes
disciplines = df_medals['discipline_title'].unique()
print(len(disciplines), disciplines[0:5])

In [ ]:
# Join the two dataframes with 'athlete_url'
df = pd.merge(df_medals, df_athletes, on='athlete_url')
df.head()

In [ ]:
# rename 'game_slug' into 'slug_game' (df_hosts)
df_hosts.rename(columns={'game_slug': 'slug_game'}, inplace=True)

In [ ]:
# Join the two dataframe (df and df_hosts) with 'slug_game'
df = pd.merge(df, df_hosts, on='slug_game')
df.head()

In [ ]:
# clean athlete medals

# ajouter un colone nb_gold, nb_silver, nb_bronze et les remplir en fonction de (athlete_medals)
# \n\n\n1\n\nG\n\n -> 1 gold medal
# \n\n\n1\n\nS\n\n\n\n1\n\nB\n\n -> 1 silver medal, 1 bronze medal
# supprimer les '\n' puis convertir 2G3S1B en 2 gold, 3 silver, 1 bronze
# test = df['athlete_medals'][2]
# test = test.replace('\n', '')
# # regex to find all (*d)G in the test
# gold = re.findall(r'(\d)G', test)
# silver = re.findall(r'(\d)S', test)
# bronze = re.findall(r'(\d)B', test)

# gold = int(gold[0]) if len(gold) > 0 else 0
# silver = int(silver[0]) if len(silver) > 0 else 0
# bronze = int(bronze[0]) if len(bronze) > 0 else 0

import re

def extract_medals(athletes_medals_str) -> str:
    if (athletes_medals_str == None):
        return 0, 0, 0, 0

    athletes_medals_str = athletes_medals_str.replace('\n', '')
    gold = re.findall(r'(\d)G', athletes_medals_str)
    silver = re.findall(r'(\d)S', athletes_medals_str)
    bronze = re.findall(r'(\d)B', athletes_medals_str)

    gold = int(gold[0]) if len(gold) > 0 else 0
    silver = int(silver[0]) if len(silver) > 0 else 0
    bronze = int(bronze[0]) if len(bronze) > 0 else 0
    total = gold + silver + bronze

    return total, gold, silver, bronze


In [ ]:
df['nb_medals'] = df['athlete_medals'].apply(lambda x: extract_medals(x)[0])
df['nb_gold'] = df['athlete_medals'].apply(lambda x: extract_medals(x)[1])
df['nb_silver'] = df['athlete_medals'].apply(lambda x: extract_medals(x)[2])
df['nb_bronze'] = df['athlete_medals'].apply(lambda x: extract_medals(x)[3])

print(len(df))

In [ ]:
# remove 'athlete_medals' column
df.drop(columns=['athlete_medals'], inplace=True)
# remove 'bio' column
df.drop(columns=['bio'], inplace=True)
# remove country_code, country_3_letter_code
df.drop(columns=['country_code', 'country_3_letter_code'], inplace=True)
# remove 'participant_title'
df.drop(columns=['participant_title'], inplace=True)
# remove 'medal_type'
df.drop(columns=['medal_type'], inplace=True)
# remove slug_game
df.drop(columns=['slug_game'], inplace=True)
# remove first column (	Unnamed: 0	)
df.drop(columns=['Unnamed: 0'], inplace=True)
# remove 'athlete_url'
df.drop(columns=['athlete_url'], inplace=True)
# remove 'athlete_full_name_y'
df.drop(columns=['athlete_full_name_y'], inplace=True)

# rename 'athlete_full_name_x' to 'athlete_full_name'
df.rename(columns={'athlete_full_name_x': 'athlete_full_name'}, inplace=True)

# drop Winter
df = df[df['game_season'] != 'Winter']

# drop 'game_season'
df.drop(columns=['game_season'], inplace=True)

# drop 'game_name'
df.drop(columns=['game_name'], inplace=True)

df.head()

In [ ]:
print(len(df))

# A faire
* Trier le dataframe par pays (faire un check des pays et regrouper en fonctions de leurs differents noms dans le temps)
* Trier le dataframe par dicipline
* Trier par athlètes ?

In [ ]:
df.to_excel('olympic_data.xlsx', index=False)